# Manual Measurement #

## Prerequisites ##

In [0]:
!pip install rasterio

In [0]:
import copy

import PIL.Image

import numpy as np
import rasterio as rio

## Mount (colab only) ##

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Load Imagery, Predictions, and "Ground Truth" ##

In [0]:
imagery_path = '/content/gdrive/My Drive/data/imagery.tif'
imagery_nd = 0 # nodata value of 0
prediction_path = '/content/gdrive/My Drive/data/prediction.tif'
truth_path = '/content/gdrive/My Drive/data/truth.tif'
truth_nd = 0 # nodata value 0
truth_gte = 6 # "yes" values >= 6

with rio.open(imagery_path, 'r') as ds:
    imagery_profile = copy.deepcopy(ds.profile)
    imagery_data = ds.read()

with rio.open(prediction_path, 'r') as ds:
    prediction_profile = copy.deepcopy(ds.profile)
    prediction_data = ds.read()

with rio.open(truth_path, 'r') as ds:
    truth_profile = copy.deepcopy(ds.profile)
    truth_data = ds.read()

In [0]:
assert(imagery_profile.get('width') == prediction_profile.get('width'))
assert(prediction_profile.get('width') == truth_profile.get('width'))
assert(imagery_profile.get('height') == prediction_profile.get('height'))
assert(prediction_profile.get('height') == truth_profile.get('height'))
width = imagery_profile.get('width')
height = imagery_profile.get('height')

## Compute Spectral Discrimination Index ##

In [0]:
if imagery_nd is not None:
    imagery_mask = (imagery_data[0] != imagery_nd).astype(np.uint8)
else:
    imagery_mask = np.ones((height, width), dtype=np.uint8)
if truth_nd is not None:
    truth_mask = (truth_data[0] != truth_nd).astype(np.uint8)
else:
    truth_mask = np.ones((height, width), dtype=np.uint8)

valid_mask = imagery_mask * truth_mask
mask = truth_data >= truth_gte


In [0]:
class_no = np.extract((mask != 1) * valid_mask, prediction_data)
class_yes = np.extract(mask * valid_mask, prediction_data)

In [0]:
mu_no = np.mean(class_no)
sigma_no = np.std(class_no)
mu_yes = np.mean(class_yes)
sigma_yes = np.std(class_yes)

In [0]:
print('sdi = {}'.format(np.abs(mu_no - mu_yes)/(sigma_no + sigma_yes)))

- A score of more than 3 is "excellent"
- A score of between 3 and 1 is "good"
- A score of less than 1 is "not so good"

## Visualize ##

In [0]:
predict_min = np.min(prediction_data)
predict_max = np.max(prediction_data)
pred_uint8 = (np.clip((prediction_data - predict_min)/(predict_max - predict_min), 0.0, 1.0) * 255).astype(np.uint8)[0]

In [0]:
PIL.Image.fromarray(pred_uint8)

In [0]:
nos = np.abs(prediction_data - mu_no)
nos_min = np.min(nos)
nos_max = np.max(nos)
nos_uint8 = (np.clip((nos - nos_min)/(nos_max - nos_min), 0.0, 1.0) * 255).astype(np.uint8)[0]
nos_uint8 = 255 - nos_uint8

In [0]:
PIL.Image.fromarray(nos_uint8)

In [0]:
yeses = np.abs(prediction_data - mu_yes)
yeses_min = np.min(yeses)
yeses_max = np.max(yeses)
yeses_uint8 = (np.clip((yeses - yeses_min)/(yeses_max - yeses_min), 0.0, 1.0) * 255).astype(np.uint8)[0]
yeses_uint8 = 255 - yeses_uint8

In [0]:
PIL.Image.fromarray(yeses_uint8)

In [0]:
PIL.Image.fromarray(np.stack([nos_uint8, pred_uint8, yeses_uint8], axis=2))

# Batch Measurement #

### Step 1 ###

Clone the [`geotrellis/deeplab-nlcd`](https://github.com/geotrellis/deeplab-nlcd) repository to a local directory.

Type
```
git clone git@github.com:geotrellis/deeplab-nlcd.git
```
or similar.

### Step 2 ###

Enter the root of the repository directory.

Type
```
cd deeplab-nlcd
```
or similar.

### Step 3 ###

Start a docker container with the needed dependencies.

Type
```
docker run -it --rm -w /workdir -v $(pwd):/workdir -v $HOME/Desktop:/desktop --runtime=nvidia jamesmcclain/aws-batch-ml:9 bash
```
or similar.  This sample command line will mount the local directory `~/Desktop/` which is assumed to contain the imagery on which we wish to work.  We will see later that it is also possible to use imagery on S3.

### Step 4 ###

Now calculate the spectral discrimination index using imagery, predictions, and "ground truth".

Type
```
python3 /workdir/python/local/sdi.py --imagery /desktop/imagery/image*.tif --imagery-nd 0 --predictions  /desktop/predictions/cheaplab/image*tif --truth //desktop/labels/image*.tif --truth-gte 6 --truth-nd 0
```
or similar.

Note that `~/Desktop/imagery/` is assumed to contain the imagery (files with names matching the pattern `image*.tif`), the directory `~/Desktop/predictions/cheaplab/` is assumed to contain the predictions (files with names matching the pattern `image*.tif`), and the directory `~/Desktop/labels/` is assumed to contain the "ground truth" labels (files with names matching the pattern `image*.tif`).
